# Assignment 5 - Issue #854

Fernando Mendoza | Adrea Pezo | Michel Cotrina | Estefanny Gil | Armando Ore

The script should not give an error. Any mistake will be consider as 0.
<br>We want an excel file with all the available presidential elections in this webpage.
https://infogob.jne.gob.pe/Eleccion
<br>The final excel should have these 3 columns 
(Elecciones | ORGANIZACIÓN POLÍTICA | TOTAL VOTOS)

In [27]:
#!pip install selenium
#!pip install webdriver-manager

## a. Setup for Chrome only

In [97]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time 
import pandas as pd

# Path for google Chrome
chrome_driver_path = r'chromedriver-win64/chromedriver.exe'
# Configue the service for google chrome
service = Service(executable_path=chrome_driver_path)
# Start the WebDriver for google chrome
driver = webdriver.Chrome(service=service)

## b. Setup for FireFox only

In [75]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time 
import pandas as pd

# Path for geckodriver (Firefox)
gecko_driver_path = r'geckodriver-v0.34.0-win64\geckodriver.exe'

# Configure the service for Firefox
service = FirefoxService(executable_path=gecko_driver_path)

# Start the WebDriver for Firefox
driver = webdriver.Firefox(service=service)


## c. Scrip for "https://infogob.jne.gob.pe/Eleccion"

In [99]:
# Maximize window
driver.maximize_window()

# Base URL for the web scraping. Open a new window
url_base = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url_base)


In [101]:
# Display "Tipo de proceso" options
opt_Presidenciales = driver.find_element(By.XPATH,
                                         '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[1]')
opt_Presidenciales.click()

# Select "ELECCIONES PRESIDENCIALES"
opt_Presidenciales_2 = driver.find_element(By.XPATH, 
                                           '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
opt_Presidenciales_2.click()

In [81]:
#In case we get a vacuum, we print as follows:


try:
    # Wait until the options are present
    options_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="IdEleccion"]/option'))
    )
    
    # Print the internal HTML of each option for debugging
    for option in options_elements:
        print(option.get_attribute('outerHTML'))

except Exception as e:
    print(f"Error: {e}")

<option value="7x8igXWeaZ8=8X" selected="selected">[SELECCIONE]</option>
<option value="89S3sk+pJI0=Sk">PRESIDENCIAL 2021 - 2DA VUELTA</option>
<option value="y0NC1IW14uw=NI">PRESIDENCIAL 2021</option>
<option value="7p675u8oH7Q=6u">PRESIDENCIAL 2016 - 2DA VUELTA</option>
<option value="LY8odlbsty4=8l">PRESIDENCIAL 2016</option>
<option value="0xj1o+1NixM=j+">PRESIDENCIAL 2011 - 2DA VUELTA</option>
<option value="24JPDKD/FrU=JK">PRESIDENCIAL 2011</option>
<option value="vJqK+o5B88E=qo">PRESIDENCIAL 2006 - 2DA VUELTA</option>
<option value="bgnkVFt+VRk=nF">PRESIDENCIAL 2006</option>
<option value="3F8G0syb5XE=8s">PRESIDENCIAL 2001 - 2DA VUELTA</option>
<option value="+kvFvkuAfuw=vk">PRESIDENCIAL 2001</option>
<option value="r+6hROwHBIQ=6O">PRESIDENCIAL 2000 - 2DA VUELTA</option>
<option value="yny1iTkPazM=yT">PRESIDENCIAL 2000</option>
<option value="8MCLSVVX8ug=CV">PRESIDENCIAL 1995</option>
<option value="E1U9NQHS0Xs=UQ">PRESIDENCIAL 1990 - 2DA VUELTA</option>
<option value="gsg4hnUOp

3. Go to Candidatos y Resultados

In [85]:
#function that extracts a df saving it into a list
def extract_df():
    # Find the element containing the table and get its HTML content
    df = driver.find_element(By.XPATH, '//*[@id="CandidatosResultados"]/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    df_1 = StringIO(df)
    # Pd to read HTML converting into df
    data = pd.read_html(df_1)
    # Return the first DataFrame found
    return data[0]

In [87]:
#check results and move to the next path
def next_button(i):
    try:
         # Wait a maximum wait time of 10 seconds
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="CandidatosResultados"]/div/div[1]/div[2]/div[2]/div[4]/div[2]/ul/li[{i}]/a'))
        )
        # Click next
        next_button.click()
        return True
        # Return False if there's a timeout
    except (TimeoutException, ElementClickInterceptedException):
        return False